In [23]:
from PIL import Image
import urllib.request
import io
from skimage import io
from ipywidgets import Image
import pandas as pd
import time 
import ast
from indeed_functions import *
import requests

In [15]:
def get_address(query):
        base = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input='
        locationbias= '42.3314584645106, -71.1039191294055'
        key = "YOUR API KEY"
        query = query
        fields = 'formatted_address,name,geometry'
        inputtype = 'textquery'
        
        url_search = base + query + '&inputtype=' + inputtype + '&fields=' + fields + '&locationbias=circle:2000@' + locationbias + '&key=' + key
        my_dict = requests.get(url_search).json()
        return my_dict

In [2]:
def get_commute(dest, origin):
    
    base = 'https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial&'
    origin = 'origins=' + origin + '&'
    dest_str = 'destinations=' + dest + '&'
    api_key = 'key=YOUR API KEY'
    
    url_search = base + origin + dest_str + api_key
    my_dict = requests.get(url_search).json()
    return my_dict

In [21]:
def indeed_scraper(results, city, job_title):
    num = -1
    file = 1
    SKIPPER = 0 # Skips this many of the first jobs
    col_list = ['title', 'company', 'location', 'summary', 'salary', 'link', 'full_text','EA_status', 'id']
    job_df = pd.DataFrame(columns=col_list, index=range(0,results))
    
    if(file > SKIPPER):
        # for results
        for start in range(0, results, 10):
            temp_url = 'http://www.indeed.com/jobs?q=' + job_title +'&l=' + str(city) + '&start=' + str(start)+'&sort=date'
            page = requests.get(temp_url) 
            time.sleep(1)  
            soup = get_soup(page.text)
            divs = soup.find_all(name="div", attrs={"class":"row"})
            # if results exist
            if(len(divs) == 0):
                break
            
            for div in divs:
                if (extract_date(div) in ['Just Posted', 'Today', '1 day ago', '2 days ago']):
                    #specifying row num for index of job posting in dataframe
                    num = num + 1
                    job_df.loc[num, 'pulldate'] = str(time.strftime("%m-%d-%Y_%I-%M_%p", time.localtime(time.time())))
                    job_df.loc[num, 'link'] = extract_link(div)
                    job_df.loc[num, 'title'] = extract_job_title(div)
                    job_df.loc[num, 'company'] = extract_company(div)
                    job_df.loc[num, 'summary'] = extract_summary(div)
                    job_df.loc[num, 'salary'] = extract_salary(div)
                    job_df.loc[num, 'location'] = extract_location(div)
                    job_df.loc[num, 'EA_status'] = extract_easyapply(div)
                    job_df.loc[num, 'id'] = div['id']
    job_df = job_df.dropna(how='all')
    companies = job_df['company']
    ids = job_df['id']
    comm_df = pd.DataFrame(columns=['company', 'place', 'address', 'distance', 'duration'])
    
    for index, company in companies.items():

        text_dict = get_address(company)
        if text_dict['status'] == 'ZERO_RESULTS':
            print(company + ' has 0 results in get_address.')
            continue
        else:
        address = text_dict['candidates'][0]['formatted_address']
        place = text_dict['candidates'][0]['name']    
        res_lat_lng = str(text_dict['candidates'][0]['geometry']['location']['lat']) + ',' + str(text_dict['candidates'][0]['geometry']['location']['lng'])

        comm = get_commute(dest=res_lat_lng, origin='8 Oswald Street')
        distance = (comm['rows'][0]['elements'][0]['distance']['text'])
        duration = (comm['rows'][0]['elements'][0]['duration']['text'])

        comm_df.loc[index, 'company'] = company
        comm_df.loc[index, 'place'] = place
        comm_df.loc[index, 'address'] = address
        comm_df.loc[index, 'distance'] = distance
        comm_df.loc[index, 'duration'] = duration
        
    comm_df['id'] = ids
    fin_df = job_df.merge(comm_df, how = 'outer', left_on='id', right_on='id').drop('company_y',1)
    file = file + 1
    return fin_df.rename({'company_x':'company'})

In [22]:
fetch = indeed_scraper(50, job_title= 'data+science', city='boston')
fetch

Galaxy AI has 0 results in get_address.


,title,company_x,location,summary,salary,link,full_text,EA_status,id,pulldate,place,address,distance,duration
0,Associate Data Center Engineer Oath – IC1,Oath Inc,"Quincy, MA",Oath’s Data Center Operations team includes a ...,Unspecified,http://www.indeed.com/rc/clk?jk=5993c280f5979a...,NaN,False,p_5993c280f5979a4b,10-22-2018_10-10_PM,AOL,"31 St James Ave, Boston, MA 02116, USA",2.4 mi,14 mins
1,Biacore Analyst (Research Associate),Sartorius Corporation,"Cambridge, MA","After an initial period of training, the analy...",Unspecified,http://www.indeed.com/rc/clk?jk=1ba4045cf1d6d5...,NaN,False,p_1ba4045cf1d6d542,10-22-2018_10-10_PM,Sartorius Corporation,"5, Orville Dr suite 200, Bohemia, NY 11716, USA",244 mi,4 hours 5 mins
2,Chief Machine Learning Scientist,Galaxy AI,"Cambridge, MA 02138 (West Cambridge area)",Lead data science efforts in one of Inc. AI Sc...,Unspecified,http://www.indeed.com/company/Galaxy-AI/jobs/C...,NaN,True,p_1cf7ab8408666cea,10-22-2018_10-10_PM,NaN,NaN,NaN,NaN
3,Data Scientist – Applied Research,Mitre Corporation,"Bedford, MA",PhD in data science or a related field (e.g. E...,Unspecified,http://www.indeed.com/rc/clk?jk=77e7c4f666cf20...,NaN,False,p_77e7c4f666cf20b8,10-22-2018_10-10_PM,MITRE,"202 Burlington Rd, Bedford, MA 01730, USA",22.6 mi,33 mins
4,Research Scientist – Natural Product Downstrea...,Conagen,"Bedford, MA",Ability to decipher NMR data for structural ch...,Unspecified,http://www.indeed.com/rc/clk?jk=346d9386d2ca66...,NaN,True,p_346d9386d2ca6698,10-22-2018_10-10_PM,Conagen,"15 Deangelo Dr, Bedford, MA 01730, USA",21.7 mi,40 mins
5,Research Data Scientist,Amazon.com,"Cambridge, MA",Ensure data quality throughout all stages of a...,Unspecified,http://www.indeed.com/rc/clk?jk=fcd51fe3b419ed...,NaN,False,p_fcd51fe3b419ed63,10-22-2018_10-10_PM,Amazon Locker - Symphony,"15 Westland Ave, at Whole Foods, Boston, MA 02...",1.6 mi,9 mins
6,Quantitative Researcher- Machine Learning,Akuna Capital,"Boston, MA",Masters or PhD in a technical field- Engineeri...,Unspecified,http://www.indeed.com/rc/clk?jk=9d47575b526f97...,NaN,False,p_9d47575b526f97df,10-22-2018_10-10_PM,Akuna Capital,"Cna Center, 333 S Wabash Ave floor 26, Chicago...",982 mi,14 hours 52 mins
7,Machine Learning Scientist,Amazon.com,"Cambridge, MA",You will leverage Amazon’s heterogeneous data ...,Unspecified,http://www.indeed.com/rc/clk?jk=bf307a9450c0a0...,NaN,False,p_bf307a9450c0a006,10-22-2018_10-10_PM,Amazon Locker - Symphony,"15 Westland Ave, at Whole Foods, Boston, MA 02...",1.6 mi,9 mins
8,Senior Machine Learning Engineer,Pluralsight,"Boston, MA",In Computer Science or relevant quantitative s...,Unspecified,http://www.indeed.com/rc/clk?jk=bc9277f872ff23...,NaN,False,p_bc9277f872ff23c0,10-22-2018_10-10_PM,Pluralsight,"60 State St suite 800, Boston, MA 02109, USA",5.0 mi,20 mins
9,Senior Manager Research Analyst,Manulife,"Boston, MA 02210 (South Boston area)",Are you looking for unlimited opportunities to...,Unspecified,http://www.indeed.com/rc/clk?jk=04aa6effc7edf4...,NaN,False,p_04aa6effc7edf4d6,10-22-2018_10-10_PM,Manulife Asset Management,"197 Clarendon St, Boston, MA 02116, USA",2.3 mi,14 mins
